In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation
from keras.layers import BatchNormalization
from keras.utils import np_utils

from tensorflow.keras.applications import ResNet50

In [2]:
(X_train, Y_train),(X_test, y_test) = cifar10.load_data()
(X_train.shape, Y_train.shape),(X_test.shape, y_test.shape)

170508288/170498071 [==============================] - 2s 0us/step


(((50000, 32, 32, 3), (50000, 1)), ((10000, 32, 32, 3), (10000, 1)))

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 4, test_size = 0.3)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((35000, 32, 32, 3), (15000, 32, 32, 3), (35000, 1), (15000, 1))

In [5]:
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)
y_val = to_categorical(y_val,10)

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
      )

test_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
    )

val_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
    )

In [7]:
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(X_test)

In [8]:
from keras.callbacks import ReduceLROnPlateau

In [9]:
lrr = ReduceLROnPlateau(monitor = "val_accuracy", factor = 0.01, patience=3, min_lr=1e-5)

In [13]:
model = Sequential()
model.add(ResNet50(weights = "imagenet", include_top = True))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dense(10,activation = "softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 256)               256256    
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 25,895,538
Trainable params: 25,842,418
Non-trainable params: 53,120
_________________________________________________________________


In [14]:
from tensorflow.keras.optimizers import Adam
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = "accuracy")

In [15]:
model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size = 128), 
    epochs = 5, 
    validation_data = val_generator.flow(x_val,y_val, batch_size = 128),
    callbacks = [lrr],
    verbose = 1 
    )

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


274/274 [==============================] - 61s 193ms/step - loss: 2.1523 - accuracy: 0.1901 - val_loss: 2.9642 - val_accuracy: 0.1012 - lr: 0.0010
Epoch 2/5
274/274 [==============================] - 53s 192ms/step - loss: 1.9468 - accuracy: 0.2613 - val_loss: 2.1047 - val_accuracy: 0.2539 - lr: 0.0010
Epoch 3/5
274/274 [==============================] - 52s 188ms/step - loss: 1.7866 - accuracy: 0.3326 - val_loss: 1.8055 - val_accuracy: 0.3368 - lr: 0.0010
Epoch 4/5
274/274 [==============================] - 52s 188ms/step - loss: 1.6761 - accuracy: 0.3777 - val_loss: 2.2057 - val_accuracy: 0.2931 - lr: 0.0010
Epoch 5/5
274/274 [==============================] - 52s 188ms/step - loss: 1.5617 - accuracy: 0.4275 - val_loss: 1.9382 - val_accuracy: 0.3431 - lr: 0.0010
